# E-commerce Web Scraping

## Business Problem 📝💹

En este proyecto trabajamos para una compañía (Whirpool) de e-commerce que se prepara para un fin de semana de promociones. Como tarea nos fue asignado averigüar los precios que maneja la competencia de algunos productos especiales. Nuestro objetico es **determinar el precio mínimo** que maneja la competencia para cada producto para así poder igualarlo. 

Stakeholders: 
* Marketing manager

Recibimos:
* Un archivo de excel (Products_and_comp.xlsx) con una lista de los productos sobre los cuales debemos hacer web scraping.
* Los nombres de las compañías que son competencia y sobre las cuales debemos ingresar a su portal de e-commerce.

Entregable:
* Un pandas dataFrame con los precios y los URL de los productos para cada compañía de la competencia.

In [1]:

import pandas as pd
import numpy as np
import re
import time

# import requests
# from urllib.request import Request, urlopen
# from bs4 import BeautifulSoup

# Creamos un entorno virtual y usamos las siguientes librerias:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager


In [2]:
excel = pd.read_excel('Products_and_comp.xlsx')
excel.shape

(424, 35)

In [3]:
excel

,Material,Marca,Sub-Categoría,Liverpool,Liverpool URL,Coppel,Coppel URL,Home Depot,Home Depot URL,Costco,...,REAMI,REAMI URL,Soriana,Soriana URL,La Unica,La Unica URL,Cimaco,Cimaco URL,Cyberpuerta,Cyberpuerta Url
0,WC11016Q,WHIRLPOOL,Congeladores,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KSM150PSER,KITCHENAID,Batidoras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KP26M1XOB,KITCHENAID,Batidoras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WK5053Q,WHIRLPOOL,Despachadores,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WM1807B,WHIRLPOOL,Hornos de Microondas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,AT1130F,ACROS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,AT1330D,ACROS,Dos puertas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421,WA4226Q,WHIRLPOOL,Frío/Calor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,AM1807B,ACROS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Identificadores de los productos
productos = excel['Material']

In [5]:
# Obtenemos la competencia
competencia = excel.columns[3::2]
competencia

Index(['Liverpool', 'Coppel', 'Home Depot', 'Costco', 'Sears', 'Elektra',
       'Walmart', 'Sam's', 'Suburbia', 'Palacio', 'Famsa', 'REAMI', 'Soriana',
       'La Unica', 'Cimaco', 'Cyberpuerta'],
      dtype='object')

## Liverpool Mex

Empezamos haciendole Web Scraping a la primera compañía de la competencia. La metodología es la siguiente:
1) Seleccionamos el boton de la barra de búsquedas
2) Escribimos los identificadores de los productos
3) De los resultados obtenemos: precio, nombre y link. (Solo tomamos el precio final, i.e. el de venta y no al amnterior a las ´promociones).
4) Automatizamos con selenium y un for que pase por cada identificador
5) Generamos un csv con todos los resultados.


URL = 'https://www.liverpool.com.mx/tienda/home'

In [ ]:
datos = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()
driver.get("https://www.liverpool.com.mx/tienda/home")

for i in range(len(productos)):
    try:
        search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//input[@class='form-control search-bar plp-no__results']")))
        #search_bar = driver.find_element(By.CLASS_NAME, "form-control search-bar plp-no__results")
        
        #Here we delete the previous search
        search_bar.send_keys(Keys.CONTROL, 'a')
        time.sleep(0.5) # This page needs some time to load.
        search_bar.send_keys(Keys.BACKSPACE)
        
        #We write our query
        search_bar.send_keys(productos[i])
        search_bar.send_keys(Keys.ENTER)
        
        try:       
            time.sleep(2) # To reload the poge needs aprox this time
            xpath_titulo = "//h1[@class='a-product__information--title']"
            titulo = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath_titulo)))
            titulo = driver.find_element(By.XPATH, xpath_titulo)
            titulo = [titulo.text]
            
            
            xpath_precio = "//p[@class = 'a-product__paragraphDiscountPrice m-0 d-inline ']"
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath_precio)))
            precio = driver.find_element(By.XPATH, xpath_precio)
            precio = [precio.text]
            
            
            get_url = driver.current_url
            link = [str(get_url)]

            # Añadimos el producto con datos np.Nan
            df = pd.DataFrame({'titles': titulo, \
                               'prices': precio, \
                               'links': link, \
                               'id': [productos[i]]}) 
            datos.append(df)
            print('Encontrado:' + str(productos[i]))

        except:
            # Si este xpath existe es poque la búsqueda devolvió nada.
            nada_xpath = "//div[@class='o-content__noResultsNullSearch']//p[@class = 'o-nullproduct-query'][1]"
            nada = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, nada_xpath))) 
            # Añadimos el producto con datos np.Nan
            df = pd.DataFrame({'titles': [np.nan], \
                               'prices': [np.nan], \
                               'links': [np.nan], \
                               'id': [productos[i]]}) 
            datos.append(df)
    except:
        print('Error in buttom in pos ' + str(productos[i]))

driver.quit() 
datos_final = pd.concat(datos, ignore_index = True)
datos_final.to_csv('scrapped_csv/Liverpool.csv')

[WDM] - Downloading: 100%|████████████| 6.58M/6.58M [00:05<00:00, 1.33MB/s]


Encontrado:WT1431A
Encontrado:WFR3200D
Encontrado:8MWTW2231WJB
Encontrado:WWR5000S
Encontrado:WH7610B
Encontrado:7MWFC5090KC
Encontrado:WA5059Q
Encontrado:7KHBV53MOB
Encontrado:AH5500S
Encontrado:WMC30516HZ
Encontrado:7MMEDC300DW
Encontrado:KFC3516CU
Encontrado:7MMVWC565FW
Encontrado:7MWET4027HW
Encontrado:7KSB1325MPA
Error in buttom in pos KHM7210CU
Encontrado:8MWTW2031WJM
Encontrado:WRE57BKTWW
Encontrado:WK5915BD
Encontrado:WH6010S
Encontrado:7KHBV53MDG
Encontrado:7MWGT4027HW
Encontrado:8MWTW2241WJB
Encontrado:WH7610D
Encontrado:MDB4949SKZ
Encontrado:ALF2053ER
Encontrado:AKT790IXL
Encontrado:810-0021
Encontrado:WW3310S
Encontrado:ALD1945JE
Encontrado:WH7610S
Encontrado:MF8000S
Encontrado:WM2811D
Encontrado:WFR5100S
Encontrado:8MWTW2041WJM
Encontrado:7MMHW6621HW
Encontrado:WT1331D
Encontrado:WER3100D
Encontrado:AF5001Z
Encontrado:WH8010D
Encontrado:WT1143K
Encontrado:WRQ551SNJZ
Encontrado:WAD2005Q
Encontrado:KP26M1XNP
Encontrado:7KHBV53MPA
Encontrado:7KSB4027MPA
Error in buttom in pos

In [99]:
liverpool = datos_final.copy()
# liverpool = pd.read_csv(/, index_col = [0])

,titles,prices,links,id
0,NaN,NaN,NaN,7MWGD1730JQ


In [ ]:
liverpool

## Coppel Mex & Home Depot
No disponible. Lo mismo sucede con Home Depot.
![Coppel](images/Coppel.png)

## Costco

In [ ]:
datos = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()
driver.get("https://www.costco.com.mx/")

search_xpath = "//div[@id='searchBoxContainer']//input[@class='search-input']"
search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, search_xpath)))
#search_bar = driver.find_element(By.CLASS_NAME, "form-control search-bar plp-no__results")

#Here we delete the previous search
search_bar.send_keys(Keys.CONTROL, 'a')
time.sleep(0.5) # This page needs some time to load.
search_bar.send_keys(Keys.BACKSPACE)

#We write our query
search_bar.send_keys(productos[i])
search_bar.send_keys(Keys.ENTER)


driver.quit() 

In [ ]:
datos = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()
driver.get("https://www.costco.com/")

for i in range(10):
    try:
        search_xpath = "//input[@class='form-control typeahead cstm-margn tt-input']"
        search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, search_xpath)))
        #search_bar = driver.find_element(By.CLASS_NAME, "form-control search-bar plp-no__results")
        
        #Here we delete the previous search
        search_bar.send_keys(Keys.CONTROL, 'a')
        time.sleep(0.5) # This page needs some time to load.
        search_bar.send_keys(Keys.BACKSPACE)
        
        #We write our query
        search_bar.send_keys(productos[i])
        search_bar.send_keys(Keys.ENTER)
        
        try:       
            time.sleep(2) # To reload the poge needs aprox this time
            xpath_titulo = "//h1[@class='a-product__information--title']"
            titulo = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath_titulo)))
            titulo = driver.find_element(By.XPATH, xpath_titulo)
            titulo = [titulo.text]
            
            
            xpath_precio = "//p[@class = 'a-product__paragraphDiscountPrice m-0 d-inline ']"
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath_precio)))
            precio = driver.find_element(By.XPATH, xpath_precio)
            precio = [precio.text]
            
            
            get_url = driver.current_url
            link = [str(get_url)]

            # Añadimos el producto con datos np.Nan
            df = pd.DataFrame({'titles': titulo, \
                               'prices': precio, \
                               'links': link, \
                               'id': [productos[i]]}) 
            datos.append(df)
            print('Encontrado:' + str(productos[i]))

        except:
            # Si este xpath existe es poque la búsqueda devolvió nada.
            nada_xpath ="//h1[@class='headline']//span[@class='ng-star-inserted']"
            nada = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, nada_xpath))) 
            # Añadimos el producto con datos np.Nan
            df = pd.DataFrame({'titles': [np.nan], \
                               'prices': [np.nan], \
                               'links': [np.nan], \
                               'id': [productos[i]]}) 
            datos.append(df)
    except:
        
        df = pd.DataFrame({'titles': [np.nan], \
                            'prices': [np.nan], \
                            'links': [np.nan], \
                            'id': [productos[i]]}) 
        datos.append(df)
                           
        
        print('Error in buttom in pos ' + str(productos[i]))

driver.quit() 
datos_final = pd.concat(datos, ignore_index = True)
datos_final.to_csv('scrapped_csv/Liverpool.csv')

## Mercado Libre Mex

Empezamos haciendole web scraping a Mercado Libre. Usamos Selenium con un driver de Chrome 108. 
Es necesario ejecutar el script _Mercado_Libre.py


URL = 'https://www.mercadolibre.com.mx/a/store/seagate'

### Test
Después de correr el script, vemos el csv final.


In [40]:
datos = []

driver = webdriver.Chrome("./chromedriver")
driver.get("https://www.mercadolibre.com.mx/")
for i in range(len(productos)):
    try:
        # search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "nav-search-input")))
        search_bar = driver.find_element(By.CLASS_NAME, "nav-search-input")
        search_bar.clear()
        search_bar.send_keys(productos[i])
        search_bar.send_keys(Keys.ENTER)

        try:
            #xpath2 = "//h2[@class='ui-search-item__title ui-search-item__group__element shops__items-group-details shops__item-title']"
            xpath = "//h2[@class='ui-search-item__title shops__item-title']"
            title_products = driver.find_elements(By.XPATH, xpath)
            title_products = [title.text for title in title_products]
            

            xpathp = "//div[@class='ui-search-price ui-search-price--size-medium shops__price']//span[@class='price-tag ui-search-price__part shops__price-part']//span[@class='price-tag-fraction']"
            price_products = driver.find_elements(By.XPATH,xpathp)
            price_products = [price.text for price in price_products]

            xpathl = "//div[@class='ui-search-item__group ui-search-item__group--title shops__items-group']//a[1]"
            links = driver.find_elements(By.XPATH, xpathl)
            links = [link.get_attribute("href") for link in links]

            if len(links) == 0 or len(price_products) == 0 or len(title_products) == 0:
                xpath2 = "//h2[@class='ui-search-item__title ui-search-item__group__element shops__items-group-details shops__item-title']"
                #xpath = "//h2[@class='ui-search-item__title shops__item-title']"
                title_products = driver.find_elements(By.XPATH, xpath2)
                title_products = [title.text for title in title_products]
                

                xpathp = "//div[@class='ui-search-price ui-search-price--size-medium shops__price']//span[@class='price-tag ui-search-price__part shops__price-part']//span[@class='price-tag-fraction']"
                price_products = driver.find_elements(By.XPATH,xpathp)
                price_products = [price.text for price in price_products]

                xpathl = "//div[@class='ui-search-result__wrapper shops__result-wrapper']//div[@class = 'ui-search-result__image shops__picturesStyles']//a[1]"
                links = driver.find_elements(By.XPATH, xpathl)
                links = [link.get_attribute("href") for link in links]
                if len(links) == len(price_products) and len(price_products) == len(title_products) and len(title_products):
                    products = {'titles': title_products,  'prices': price_products, 'links': links }
                    df = pd.DataFrame(products)
                    df['id'] = productos[i]
                    datos.append(df)

            elif len(links) == len(price_products) and len(price_products) == len(title_products) and len(title_products):
                products = {'titles': title_products,  'prices': price_products, 'links': links }
                df = pd.DataFrame(products)
                df['id'] = productos[i]
                datos.append(df)
            else:
                df = pd.DataFrame({'titles': [np.nan], 'prices': [np.nan], 'links': [np.nan], 'id': productos[i]})
                print('Product Not Available:' + str(productos[i]) + ' pos ' + str(i))
                datos.append(df)
            
        except:
            df = pd.DataFrame({'titles': [np.nan], 'prices': [np.nan], 'links': [np.nan], 'id': productos[i]})
            print('No link for product:' + str(productos[i] + 'pos ' + str(i)))
            datos.append(df)
    except:
        print('Error in buttom in pos ' + str(i))

driver.quit() 
datos_final = pd.concat(datos, ignore_index = True)
datos_final.to_csv('scrapped_csv/Mercado_Libre.csv')

C:\Users\diego\AppData\Local\Temp\ipykernel_6740\3717242840.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver")


No link for product:WM1807Bpos 4
Error in buttom in pos 5
Error in buttom in pos 6
Error in buttom in pos 7
Error in buttom in pos 8
Error in buttom in pos 9
Error in buttom in pos 10
Error in buttom in pos 11
Error in buttom in pos 12
Error in buttom in pos 13
Error in buttom in pos 14
Error in buttom in pos 15
Error in buttom in pos 16
Error in buttom in pos 17
Error in buttom in pos 18
Error in buttom in pos 19
Error in buttom in pos 20
Error in buttom in pos 21
Error in buttom in pos 22
Error in buttom in pos 23
Error in buttom in pos 24
Error in buttom in pos 25
Error in buttom in pos 26
Error in buttom in pos 27
Error in buttom in pos 28
Error in buttom in pos 29
Error in buttom in pos 30
Error in buttom in pos 31
Error in buttom in pos 32
Error in buttom in pos 33
Error in buttom in pos 34
Error in buttom in pos 35
Error in buttom in pos 36
Error in buttom in pos 37
Error in buttom in pos 38
Error in buttom in pos 39
Error in buttom in pos 40
Error in buttom in pos 41
Error in b

KeyboardInterrupt: 

In [8]:
resultados_test1 = pd.read_csv('test_ML.csv', index_col = [0])
resultados_test1

,titles,prices,links,id
0,Canastilla Cesta Reja Organizdor Congelador Wh...,449,https://articulo.mercadolibre.com.mx/MLM-15094...,WC11016Q
1,Congelador horizontal Whirlpool WC11016Q blanc...,"13,990",https://www.mercadolibre.com.mx/congelador-hor...,WC11016Q
2,Congelador Horizontal Whirlpool Wc11016q Blanc...,"2,499",https://articulo.mercadolibre.com.mx/MLM-16796...,WC11016Q
3,Congelador 11 Pies Whirlpool Wc11016q Blanco,"13,990",https://articulo.mercadolibre.com.mx/MLM-66872...,WC11016Q
4,Congelador Xpert Energy Saver 300.00 L / 11 P³...,"16,777",https://articulo.mercadolibre.com.mx/MLM-55919...,WC11016Q
5,Congelador Whirlpool 16 Pies Cúbicos Blanco,"12,000",https://articulo.mercadolibre.com.mx/MLM-15711...,WC11016Q


In [9]:
df = pd.read_csv('scrapped_csv/Mercado_Libre.csv', index_col = [0])
df

,titles,prices,links,id
0,Canastilla Cesta Reja Organizdor Congelador Wh...,449,https://articulo.mercadolibre.com.mx/MLM-15094...,WC11016Q
1,Congelador horizontal Whirlpool WC11016Q blanc...,"13,990",https://www.mercadolibre.com.mx/congelador-hor...,WC11016Q
2,Congelador 11 Pies Whirlpool Wc11016q Blanco,"13,990",https://articulo.mercadolibre.com.mx/MLM-66872...,WC11016Q
3,Congelador Xpert Energy Saver 300.00 L / 11 P³...,"16,777",https://articulo.mercadolibre.com.mx/MLM-55919...,WC11016Q
4,Congelador Whirlpool 16 Pies Cúbicos Blanco,"12,000",https://articulo.mercadolibre.com.mx/MLM-15711...,WC11016Q
...,...,...,...,...
100,Lavasecadora Carga Frontal 10 Kg Alta Eficienc...,"17,999",https://articulo.mercadolibre.com.mx/MLM-14208...,7MWFC5090KC
101,Lavadora semiautomática de doble tina Midea ML...,"2,700",https://www.mercadolibre.com.mx/lavadora-semia...,7MWFC5090KC
102,Lavadora automática Mabe LMA76112C blanca 16kg...,"8,449",https://www.mercadolibre.com.mx/lavadora-autom...,7MWFC5090KC
103,Lavasecadora Whirlpool 10 Kg,"10,000",https://articulo.mercadolibre.com.mx/MLM-13416...,7MWFC5090KC


In [10]:
set(productos[0:18]) - set(df.id.unique())

{'8MWTW2231WJB', 'KP26M1XOB'}

In [8]:
df.id.unique()

array(['WC11016Q', 'KSM150PSER', 'WK5053Q', 'WM1807B', 'KSM150PSWH',
       'WT1431A', 'WFR3200D', '7MWGD1730JQ', 'WS4515BS', 'WK5917B',
       'KSM100PSER', 'WM2514D', 'KFC3516ER', 'WWR5000S', 'WH7610B',
       '7MWFC5090KC'], dtype=object)

In [11]:
df[df.id == 'KSM150PSER']

,titles,prices,links,id
5,Kit Batidora De Pedestal Kitchenaid Artisan + ...,"9,879",https://articulo.mercadolibre.com.mx/MLM-15510...,KSM150PSER
6,Batidora de pedestal KitchenAid Artisan KSM150...,"12,989",https://www.mercadolibre.com.mx/batidora-de-pe...,KSM150PSER
7,Batidora de pedestal KitchenAid Artisan KSM150...,"10,655",https://www.mercadolibre.com.mx/batidora-de-pe...,KSM150PSER
8,Batidora de pedestal KitchenAid Artisan KSM150...,"8,871",https://www.mercadolibre.com.mx/batidora-de-pe...,KSM150PSER
9,Batidora De Pedestal Kitchenaid Artisan Ksm150...,"10,655",https://articulo.mercadolibre.com.mx/MLM-14407...,KSM150PSER
10,Batidora Kitchenaid Artisan 4.7 L,"11,665",https://articulo.mercadolibre.com.mx/MLM-14832...,KSM150PSER
11,Batidora Roja De Pie Kitchenaid Artisan Ksm150ps,"11,200",https://articulo.mercadolibre.com.mx/MLM-90646...,KSM150PSER
12,Batidora De Pedestal Kitchenaid Custom Ksm120e...,"8,490",https://articulo.mercadolibre.com.mx/MLM-14203...,KSM150PSER
13,Batidora Kitchenaid Artisan 4.7 L,"6,899",https://articulo.mercadolibre.com.mx/MLM-14799...,KSM150PSER
